In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

In [2]:
# all data file
with open('all_data.pkl', 'rb') as f:
    df = pickle.load(f)
df.head(2)

,node,node_name,neib_index,neib_node,0,1,2,3,4,5,...,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428
0,100001,Асбест,"[26, 61, 62, 720, 721, 722, 723, 725, 726, 728]","[100028, 100063, 100064, 101036, 101037, 10103...",1227.2242,1241.8453,1245.6560,1247.9730,1196.1394,1278.2474,...,1687.601635,1373.512108,1706.934806,1737.370083,1744.597307,1551.456345,1526.791555,1496.149319,1643.579499,1553.756279
1,100003,Травянская 1 СШ 220,"[6, 14, 78, 79, 87, 93, 729, 732, 750, 758]","[100008, 100016, 100080, 100081, 100089, 10009...",1253.7522,1266.2651,1268.6195,1271.0367,1221.8423,1307.6155,...,1740.775327,1477.795452,1752.449603,1686.724408,1576.992991,1591.783919,1570.577347,1541.377099,1685.148284,1594.559387


In [3]:
def get_neighbors(data, n):
    t = data[data['node']==n]['neib_node'].values.flatten().tolist()[0]
    return t

In [4]:
def get_prices(data, n):
    prices_cols = data.columns[4:]
    t = data[data['node']==n][prices_cols].values.flatten().tolist()
    return t

In [5]:
def forecast_prophet(data, forecast_days):
    train, test = data[:-forecast_days], data[-forecast_days:]
    m = Prophet(changepoint_prior_scale = 0.01, seasonality_prior_scale = 0.01)
    m = m.fit(train)
    future = m.make_future_dataframe(periods=forecast_days)
    forecast = m.predict(future)
    yhat = forecast['yhat'][-forecast_days:]
    rmse_res = mse(test['y'], yhat, squared=False)
    mape_res = mape(test['y'], yhat)
    return round(rmse_res, 2), round(mape_res, 3)

In [6]:
def forecast_prophet_lagged(data, forecast_days, lag):
    neibs = data.columns.to_list()[2:]
    for neib in neibs:
        data[neib] = data[neib].shift(lag)

    
    data.dropna(inplace=True)
    train, test = data[:-forecast_days], data[-forecast_days:]
    
    m = Prophet(changepoint_prior_scale = 0.01, seasonality_prior_scale = 0.01)
    for neib in neibs:
        m.add_regressor(neib)
        
    m = m.fit(train)
    
    future = m.make_future_dataframe(periods=forecast_size, include_history=False)
    for neib in neibs:
        future[neib] = test[neib].to_list() #including test nodes as they are historical - shifted
    forecast = m.predict(future)
    
    yhat = forecast['yhat'][-forecast_days:]
    rmse_res = mse(test['y'], yhat, squared=False)
    mape_res = mape(test['y'], yhat)
    return round(rmse_res, 2), round(mape_res, 3)

In [7]:
list_of_nodes = df['node'].tolist()

In [8]:
#disabiling prophet info messages
import logging
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").disabled=True

nodes_for_forecast = list_of_nodes #[100419]
dates = pd.date_range(start='2019-12-05', periods = 1429)
forecast_size=30
neighbors_number= 1

result = {'node':[], 'RMSE_1':[], 'MAPE_1':[], 'RMSE_2':[], 'MAPE_2':[]}
counter = 0
k=len(list_of_nodes)

print("working for number of neighbor:", neighbors_number)

for node in nodes_for_forecast:
    # print("working for:", node)
    prices = get_prices(df, node)
#     print(len(prices))
    curr_df = pd.DataFrame(data = {'ds':dates, 'y':prices})
    
    rmse_1, mape_1 = forecast_prophet (curr_df, forecast_size)
    #print("RMSE base:", rmse_ret_base, "MAPE base:", mape_ret_base)
    
    # add neighbors
    neighbors = get_neighbors(df, node)[:neighbors_number]
    for neighbor in neighbors:
        p = get_prices(df, neighbor)
        if (len(p) == 1429):
            curr_df[str(neighbor)] = p
    
    rmse_2, mape_2 = forecast_prophet_lagged(curr_df, forecast_size, 366)

    #print("RMSE lagg:", rmse_ret_lagged, "MAPE lagg:", mape_ret_lagged)
    result['node'].append(node)
    result['RMSE_1'].append(rmse_1)
    result['MAPE_1'].append(mape_1)
    result['RMSE_2'].append(rmse_2)
    result['MAPE_2'].append(mape_2)
    if (rmse_1 > rmse_2):
        counter +=1
    else: counter-=1
        
    k-=1   
    print(str(node) + " --> "+ str(k) + " --> "+str(counter)+"   ", end="\r")
    

working for number of neighbor: 1


In [9]:
# Save results!!

In [13]:
with open('all_data_PRF_1_neighb.pkl', 'wb') as f:
    pickle.dump(result, f)

In [11]:
df = pd.DataFrame.from_dict(result)

In [12]:
df['RMSE_1'].mean(), df['RMSE_2'].mean()

(200.11568636433043, 194.15874179264011)